<a href="https://colab.research.google.com/github/omemaakhtar/Repo-Bank/blob/main/Image_Classification_using_Transfer_learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Define paths
dataset_dir = '/content/Image-Classification-Transfer-Learning/Dataset'

# List the contents of the Dataset directory
os.listdir(dataset_dir)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Image-Classification-Transfer-Learning/Dataset'

In [1]:
!git clone https://github.com/miladfa7/Image-Classification-Transfer-Learning.git


Cloning into 'Image-Classification-Transfer-Learning'...
remote: Enumerating objects: 2102, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 2102 (delta 7), reused 6 (delta 1), pack-reused 2084 (from 1)
Receiving objects: 100% (2102/2102), 90.89 MiB | 19.38 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [2]:
import os

# Check the current working directory
print(os.getcwd())

# List the contents of the cloned repository
os.listdir('/content/Image-Classification-Transfer-Learning')


/content


['.git',
 'ResNet_image_classification.py',
 'README.md',
 'Dataset',
 'Augmentation.py',
 'ResNet_image_classification.ipynb',
 'images',
 'Json_file.py']

In [3]:
# Define the dataset path
dataset_dir = '/content/Image-Classification-Transfer-Learning/Dataset'

# List the contents of the Dataset directory
os.listdir(dataset_dir)


['test', 'training']

In [4]:
# List the contents of the training directory
training_dir = os.path.join(dataset_dir, 'training')
test_dir = os.path.join(dataset_dir, 'test')

print("Training Directory Contents:")
print(os.listdir(training_dir))

print("\nTest Directory Contents:")
print(os.listdir(test_dir))


Training Directory Contents:
['grand-piano', 'fireworks', 'airplanes', 'owl', 'waterfall', 'calculator', 'lightning', 'school-bus', 'galaxy', 'mountain-bike', 'lightbulb', 'wine-bottle', 'sword', 'bear', 'sheet-music', 't-shirt', 'skyscraper', 'kangaroo', 'laptop', 'computer-monitor']

Test Directory Contents:
['IMG_1285.jpg', 'IMG_1771.jpg', 'IMG_1494.jpg', 'IMG_501.jpg', 'IMG_1747.jpg', 'IMG_1342.jpg', 'IMG_873.jpg', 'IMG_1377.jpg', 'IMG_775.jpg', 'IMG_1444.jpg', 'IMG_1851.jpg', 'IMG_572.jpg', 'IMG_1600.jpg', 'IMG_1756.jpg', 'IMG_89.jpg', 'IMG_68.jpg', 'IMG_2007.jpg', 'IMG_1155.jpg', 'IMG_2031.jpg', 'IMG_515.jpg', 'IMG_1436.jpg', 'IMG_999.jpg', 'IMG_227.jpg', 'IMG_1512.jpg', 'IMG_524.jpg', 'IMG_539.jpg', 'IMG_938.jpg', 'IMG_34.jpg', 'IMG_1699.jpg', 'IMG_206.jpg', 'IMG_893.jpg', 'IMG_2010.jpg', 'IMG_151.jpg', 'IMG_1529.jpg', 'IMG_1643.jpg', 'IMG_1511.jpg', 'IMG_1211.jpg', 'IMG_1500.jpg', 'IMG_98.jpg', 'IMG_887.jpg', 'IMG_1966.jpg', 'IMG_280.jpg', 'IMG_1768.jpg', 'IMG_866.jpg', 'IMG_32

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define parameters
img_height, img_width = 224, 224  # Size of the input images
batch_size = 32

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Important for evaluation
)


Found 1554 images belonging to 20 classes.
Found 0 images belonging to 0 classes.


In [6]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to load and preprocess test images
def load_test_images(test_dir):
    test_images = []
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        if os.path.isfile(img_path):
            img = image.load_img(img_path, target_size=(img_height, img_width))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
            test_images.append(img_array)
    return np.vstack(test_images)

# Load test images
test_images_array = load_test_images(test_dir)


In [8]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to load and preprocess test images
def load_test_images(test_dir):
    test_images = []
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        if os.path.isfile(img_path):
            img = image.load_img(img_path, target_size=(img_height, img_width))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
            test_images.append(img_array)
    return np.vstack(test_images)

# Load test images
test_images_array = load_test_images(test_dir)

# Check the shape of the loaded test images
print("Loaded test images shape:", test_images_array.shape)


Loaded test images shape: (500, 224, 224, 3)


In [11]:
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras import layers, models

# Load the ResNet152V2 model with pre-trained weights
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model
base_model.trainable = False

# Create the input layer
inputs = layers.Input(shape=(img_height, img_width, 3))

# Pass the inputs through the base model
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(20, activation='softmax')(x)  # 20 classes

# Create the model
model = models.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet152v2 (Functional)             │ (None, 7, 7, 2048)          │      58,331,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 20)                  │          40,980 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 58,372,628 (222.67 MB)

 Trainable params: 40,980 (160.08 KB)

 Non-trainable params: 58,331,648 (222.52 MB)

In [12]:
# Define the number of epochs
epochs = 10

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_images_array,
    validation_steps=len(test_images_array) // batch_size,
    epochs=epochs
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step - accuracy: 0.4831 - loss: 1.9109

ValueError: None values not supported.

In [13]:
# Train the model without validation data for now
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs
)


Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 9s 176ms/step - accuracy: 0.9790 - loss: 0.1431
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9688 - loss: 0.1091  
Epoch 3/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 178ms/step - accuracy: 0.9940 - loss: 0.0561
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 369us/step - accuracy: 1.0000 - loss: 0.1083
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 9s 171ms/step - accuracy: 0.9981 - loss: 0.0408
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - accuracy: 1.0000 - loss: 0.0406
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 172ms/step - accuracy: 1.0000 - loss: 0.0265
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step - accuracy: 1.0000 - loss: 0.0090
Epoch 9/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 9s 180ms/step - accuracy: 1.0000 - loss: 0.0201
Epoch 10/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 334us/step - accuracy: 1.0000 - loss: 0.0077


In [14]:
# Evaluate the model on test images
test_loss, test_accuracy = model.evaluate(test_images_array, test_labels_array)

print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Loss: {test_loss:.4f}')


NameError: name 'test_labels_array' is not defined

In [17]:
print(f'Number of test images: {len(test_images_array)}')
print(f'Number of test labels: {len(test_labels)}')


Number of test images: 500
Number of test labels: 4


In [19]:
# Assuming you know how many of each class should be in the test set
class_distribution = [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]  # Adjust according to your actual distribution
class_count = len(class_distribution)

test_labels = []
for i, filename in enumerate(os.listdir(test_dir)):
    class_index = i % class_count
    test_labels.append(class_index)

# Convert to one-hot encoding if necessary
test_labels_array = np.eye(class_count)[test_labels]


In [21]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images_array, test_labels_array)

print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Loss: {test_loss:.4f}')


16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 596ms/step - accuracy: 0.0505 - loss: 1394.9641
Test Accuracy: 0.0500
Test Loss: 1389.8247


In [22]:
# Make predictions
predictions = model.predict(test_images_array)

# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Print out some predictions alongside their true labels
for i in range(5):  # Change the range to see more or fewer examples
    print(f'Image {i+1}: True label = {np.argmax(test_labels_array[i])}, Predicted label = {predicted_classes[i]}')


16/16 ━━━━━━━━━━━━━━━━━━━━ 42s 638ms/step
Image 1: True label = 0, Predicted label = 9
Image 2: True label = 1, Predicted label = 9
Image 3: True label = 2, Predicted label = 9
Image 4: True label = 3, Predicted label = 9
Image 5: True label = 4, Predicted label = 9
